# Parse Nav.no sitemap

Denne notatboken analyserer
[nav.no/sitemap.xml](https://www.nav.no/sitemap.xml). Sitemap-en beskriver alle
åpne sider på [Nav.no](https://www.nav.no) og når de siste er oppdatert. Denne
informasjonen er fin å bruke når man skal laste ned innholdet.

In [ ]:
from rich import print

In [ ]:
# Filter for å fjerne stier som vi ikke er interessert i å indeksere
PATH_FILTER = (
    "/nav-og-samfunn/statistikk",  # Bob trenger ikke å kunne svare om statistikk
    "/nav-og-samfunn/kunnskap",  # Trenger ikke å svare om kunnskapsinnhold heller
)
# Filtrer ut sidetyper som ikke gir så mye mening at Bob bruker
PAGE_TYPE_FILTER = (
    "no.nav.navno:area-page",  # Oversikt over et tema som for det meste inneholder lenker: "/helse"
    "no.nav.navno:form-intermediate-step",  # Ser ut til å bare være knapp for å sende inn søknad: "/start/soknad-aap"
    "no.nav.navno:front-page",  # De store forsidene, disse har bare lenker videre: "www.nav.no/"
    "no.nav.navno:front-page-nested",  # Oversiktsside med bare lenker: "/utbetalinger"
    "no.nav.navno:generic-page",  # Lenke side: "/send-beskjed-om-syk"
    "no.nav.navno:large-table",  # Store tabeller med statistikk
    "no.nav.navno:main-article",  # Ser ut til å være artikler/nyheter: "https://www.nav.no/no/nav-og-samfunn/samarbeid/hjelpemidler/leverandorer-av-hjelpemidler/innovasjon-og-utvikling"
    "no.nav.navno:office-page",  # Kontaktinformasjon til lokalkontor: "/kontor/nav-sor-helgeland"
    "no.nav.navno:page-list",  # Ser ut som en lenke side: "https://www.nav.no/no/nav-og-samfunn/samarbeid/leger-og-andre-behandlere/om-sykmelding-for-leger"
    "no.nav.navno:press-landing-page",  # Kontaktinformasjon for presse (bare 1 side): "/samarbeidspartner/presse"
    "no.nav.navno:section-page",  # Lenkeside (bare 2 eksempler): "/se/samegiella"
)
# Bare ta med artikler på følgende språk
LANGUAGE_INCLUDE = (
    "no",
    "nn",
)

## Leser inn sitemap fra internett

In [ ]:
import httpx

raw_sitemap = httpx.get("https://www.nav.no/sitemap.xml").raise_for_status()

In [ ]:
import xml.etree.ElementTree as ET

sitemap = ET.fromstring(raw_sitemap.text)
print(f"Antall sider i sitemap: {len(sitemap)}")

## Filtrerer ut sider

In [ ]:
import datetime

from yarl import URL

to_index: list[URL] = []

for site in sitemap:
    if not any([path in site[0].text for path in PATH_FILTER]):
        url = URL(site[0].text)
        to_index.append(url)
print(
    f"Antall sider etter filtrering: {len(to_index)} "
    f"({len(to_index) / len(sitemap):.1%} av alle sider)"
)

## Definere datastruktur

In [ ]:
from pydantic import BaseModel, HttpUrl


class SiteToIndex(BaseModel):
    """Representasjon av en nettside vi skal indeksere."""

    url: HttpUrl
    """URL til siden"""
    json_url: HttpUrl
    """URL til JSON objektet vi kan hente strukturert data"""
    last_modified: datetime.datetime | None
    """Når ble siden sist oppdatert"""

## Hent ut JSON data

In [ ]:
from rich.progress import track

index: list[SiteToIndex] = []
could_not_reach: list[URL] = []

with httpx.Client() as client:
    while to_index:
        redirects = []
        for site in track(to_index, description="Aksesserer JSON"):
            try:
                resp = client.get(
                    f"https://www.nav.no/_next/data/latest/{site.path}.json"
                ).raise_for_status()
            except httpx.HTTPStatusError:
                could_not_reach.append(site)
                continue
            data = resp.json()["pageProps"]
            if "__N_REDIRECT" in data:
                redirects.append(site.with_path(data["__N_REDIRECT"]))
                continue
            content = data["content"]
            page_type = content["type"]
            if page_type in PAGE_TYPE_FILTER:
                # Hvis siden er av en type som vi ikke trenger å indeksere så
                # hopper vi over den her
                continue
            if content["language"] not in LANGUAGE_INCLUDE:
                continue
            last_modified = content.get("modifiedTime")
            index.append(
                SiteToIndex(
                    url=str(site),
                    json_url=str(resp.request.url),
                    last_modified=last_modified,
                )
            )
        to_index.clear()
        if redirects:
            print(
                f"{len(redirects)} sider videresendte oss, må prøve å aksessere disse også"
            )
            to_index.extend(redirects)
            redirects.clear()


print(
    f"Endte med {len(index)} sider å indeksere etter filtrering på type ({len(index) / len(sitemap):.1%} av alle sider)"
)

In [ ]:
print(f"{len(could_not_reach)} sider kunne ikke nåes")

## Skriv til lokal JSON

In [ ]:
import json

from rich.prompt import Prompt

if local_file := Prompt.ask("Filnavn å skrive indeks til", default="index.json"):
    if not local_file.endswith(".json"):
        local_file += ".json"
    with open(local_file, mode="w") as fil:
        json.dump([site.model_dump(mode="json") for site in index], fil)